In [44]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import re
import plotly.express as px

In [49]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 75

#### Query nash_referrals - to_npi facility details
#### How many facilities have a NUCC grouping of 'Hospitals' - and how many NPIs do those facilities have?
(count of npis for each unique facility name)

In [91]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_grouping of 'Hospitals'

query = """
SELECT
    to_facility,
    COUNT(DISTINCT(to_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions
FROM nash_referrals
WHERE to_classification IS 'General Acute Care Hospital' 
GROUP BY to_facility
ORDER BY count_npi DESC
"""

hospitals_agg_alldays = pd.read_sql(query,db)

db.close()
hospitals_agg_alldays.shape

(16, 4)

In [96]:
print(hospitals_agg_alldays.total_transactions.sum())
hospitals_agg_alldays

1935403


,to_facility,count_npi,total_patients,total_transactions
0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",6,270573,362455
1,VANDERBILT UNIVERSITY MEDICAL CENTER,3,441266,653863
2,SAINT THOMAS WEST HOSPITAL,2,205062,247159
3,RIVERVIEW MEDICAL CENTER LLC,2,6689,13719
4,WILLIAMSON COUNTY HOSPITAL DISTRICT,1,71272,98355
5,TROUSDALE MEDICAL CENTER LLC,1,1905,4245
6,SUMNER REGIONAL MEDICAL CENTER LLC,1,46624,67847
7,"SAINT THOMAS STONES RIVER HOSPITAL, LLC",1,2463,3629
8,SAINT THOMAS RUTHERFORD HOSPITAL,1,77124,94125
9,NORTHCREST MEDICAL CENTER,1,27041,41807


In [93]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_grouping of 'Hospitals'

query = """
SELECT
    to_facility,
    COUNT(DISTINCT(to_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions
FROM nash_referrals
WHERE to_classification IS 'General Acute Care Hospital' AND average_day_wait >= 2
GROUP BY to_facility
ORDER BY count_npi DESC
"""

hospitals_agg = pd.read_sql(query,db)

db.close()
hospitals_agg.shape

(16, 4)

In [94]:
hospitals_agg.total_tansactions.sum()

,to_facility,count_npi,total_patients,total_transactions
0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",6,244956,327456
1,VANDERBILT UNIVERSITY MEDICAL CENTER,3,362528,492145
2,SAINT THOMAS WEST HOSPITAL,2,192819,230756
3,RIVERVIEW MEDICAL CENTER LLC,2,5709,12358
4,WILLIAMSON COUNTY HOSPITAL DISTRICT,1,53758,78402
5,TROUSDALE MEDICAL CENTER LLC,1,1633,3728
6,SUMNER REGIONAL MEDICAL CENTER LLC,1,40519,60174
7,"SAINT THOMAS STONES RIVER HOSPITAL, LLC",1,2372,3531
8,SAINT THOMAS RUTHERFORD HOSPITAL,1,66084,81390
9,NORTHCREST MEDICAL CENTER,1,20610,32923


### NPI level detail for hospitals

In [99]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_grouping of 'Hospitals'
#examine distinct npis for each facility name 

query = """
SELECT
    to_facility,
    to_address,
    to_grouping,
    to_classification,
    to_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions
FROM nash_referrals
WHERE to_grouping IS 'Hospitals' AND to_classification IS 'General Acute Care Hospital'
GROUP BY to_npi
ORDER BY to_facility DESC
"""

hosp_npis = pd.read_sql(query,db)

db.close()
hosp_npis.shape

(25, 7)

In [102]:
hosp_npis.to_csv('../data/hospital_npis.csv', index= False)

In [100]:
hosp_npis

,to_facility,to_address,to_grouping,to_classification,to_npi,total_patients,total_transactions
0,WILLIAMSON COUNTY HOSPITAL DISTRICT,4321 CAROTHERS PARKWAY,Hospitals,General Acute Care Hospital,1265445506,71272,98355
1,VANDERBILT UNIVERSITY MEDICAL CENTER,1601 23RD AVE S,Hospitals,General Acute Care Hospital,1558408633,1745,2598
2,VANDERBILT UNIVERSITY MEDICAL CENTER,1211 MEDICAL CENTER DRIVE,Hospitals,General Acute Care Hospital,1396882205,404973,603123
3,VANDERBILT UNIVERSITY MEDICAL CENTER,1411 W. BADDOUR PARKWAY,Hospitals,General Acute Care Hospital,1306889597,34548,48142
4,TROUSDALE MEDICAL CENTER LLC,500 CHURCH ST,Hospitals,General Acute Care Hospital,1467763458,1905,4245
5,SUMNER REGIONAL MEDICAL CENTER LLC,555 HARTSVILLE PIKE,Hospitals,General Acute Care Hospital,1447571658,46624,67847
6,SAINT THOMAS WEST HOSPITAL,2000 CHURCH ST,Hospitals,General Acute Care Hospital,1780778969,75300,90199
7,SAINT THOMAS WEST HOSPITAL,4220 HARDING RD,Hospitals,General Acute Care Hospital,1629025648,129762,156960
8,"SAINT THOMAS STONES RIVER HOSPITAL, LLC",324 DOOLITTLE RD,Hospitals,General Acute Care Hospital,1447639398,2463,3629
9,SAINT THOMAS RUTHERFORD HOSPITAL,1700 MEDICAL CENTER PKWY,Hospitals,General Acute Care Hospital,1164590386,77124,94125


### Grouping = 'Hospital Units'


In [65]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection


#select facility name where facility has to_grouping of 'Hospital Units'
#examine distinct npis for each facility name 


query = """
SELECT
    to_facility,
    to_grouping,
    to_classification,
    to_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions
FROM nash_referrals
WHERE to_grouping IS 'Hospital Units'
GROUP BY to_npi
ORDER BY to_facility DESC
"""


hosp_units = pd.read_sql(query,db)

db.close()
hosp_units.shape

(14, 6)

In [67]:
print(hosp_units.total_patients.sum())
hosp_units

10564


,to_facility,to_grouping,to_classification,to_npi,total_patients,total_transactions
0,VANDERBILT UNIVERSITY MEDICAL CENTER,Hospital Units,Psychiatric Unit,1821030842,274,328
1,VANDERBILT UNIVERSITY MEDICAL CENTER,Hospital Units,Rehabilitation Unit,1215979190,154,168
2,"TCMC MADISON-PORTLAND, INC.",Hospital Units,Psychiatric Unit,1245271741,39,110
3,SUMNER REGIONAL MEDICAL CENTER LLC,Hospital Units,Psychiatric Unit,1669793865,189,201
4,SUMNER REGIONAL MEDICAL CENTER LLC,Hospital Units,Rehabilitation Unit,1548581598,203,216
5,SAINT THOMAS WEST HOSPITAL,Hospital Units,Rehabilitation Unit,1962567222,345,361
6,"SAINT THOMAS STONES RIVER HOSPITAL, LLC",Hospital Units,Psychiatric Unit,1700267218,264,307
7,"RIVERVIEW MEDICAL CENTER, LLC",Hospital Units,Psychiatric Unit,1366703225,185,208
8,HTI MEMORIAL HOSPITAL CORPORATION,Hospital Units,Rehabilitation Unit,1639123615,2669,3032
9,HTI MEMORIAL HOSPITAL CORPORATION,Hospital Units,Psychiatric Unit,1417086596,485,580


### Verify aggregates for different hospital queries

In [105]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_grouping of 'Hospitals'
#examine distinct npis for each facility name 

query = """
SELECT
   *
FROM nucc_taxonomy
WHERE code IS '282N00000X'
LIMIT 1

"""

hosp_npis = pd.read_sql(query,db)

db.close()
hosp_npis.shape

(1, 10)

In [106]:
hosp_npis

,code,grouping,classification,specialization,definition,effective_date,deactivation_date,last_modified_date,notes,display_name
0,282N00000X,Hospitals,General Acute Care Hospital,None,An acute general hospital is an institution wh...,4/1/2002,None,None,None,General Acute Care Hospital


In [ ]:
hosp_npis

#### Verify records for individual facilities

In [68]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where

query = """
SELECT
    to_facility,
    to_grouping,
    to_classification,
    to_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions
FROM nash_referrals
WHERE to_facility LIKE '%CENTENNIAL%'
ORDER BY to_facility DESC
"""

verify = pd.read_sql(query,db)

db.close()
verify.shape

(1, 6)

In [69]:
verify

,to_facility,to_grouping,to_classification,to_npi,total_patients,total_transactions
0,"CENTENNIAL NEUROSCIENCE, LLC",Allopathic & Osteopathic Physicians,Psychiatry & Neurology,1104152859,252755,368727


#### Query nash_referrals - to_npi facility details

#### How many npis does each facility have?
(count of npis for each unique facility name)

In [33]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where

query = """
SELECT
    to_facility,
    COUNT(DISTINCT(to_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions
FROM nash_referrals
GROUP BY to_facility
ORDER BY count_npi DESC
"""

facilities = pd.read_sql(query,db)

db.close()
facilities.shape

(1057, 4)

In [34]:
facilities

,to_facility,count_npi,total_patients,total_transactions
0,TENNESSEE ONCOLOGY PLLC,13,127552,244122
1,"DIALYSIS ASSOCIATES, LLC",12,1085,6408
2,VANDERBILT UNIVERSITY MEDICAL CENTER,10,1027027,1568469
3,DIALYSIS CLINIC INC.,10,1046,6929
4,"HCA HEALTH SERVICES OF TENNESSEE, INC.",9,276229,369833
5,"UNITED NEIGHBORHOOD HEALTH SERVICES, INC",8,637,1218
6,SUMNER PHYSICIAN PRACTICES LLC,7,6992,11261
7,TENNESSEE ORTHOPAEDIC ALLIANCE PA,6,70802,130129
8,NHC-OP LP,6,1617,1995
9,HOSPITAL BASED MEDICAL SERVICES OF TENNESSEE-I PC,6,15423,16414


In [25]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
SELECT
    to_facility,
    COUNT(to_npi) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions
FROM nash_referrals
GROUP BY to_facility
ORDER BY count_npi DESC
"""
facilities = pd.read_sql(query,db)

db.close()
facilities.shape

(1057, 4)

### Acute Care Hospitals in CBSA
#### Query nash_referrals - to_npi classification & grouping details

In [4]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where

#WHERE to_classification LIKE '%Hospital%'



query = """
SELECT
    CASE WHEN to_facility LIKE '%Vanderbilt%' THEN 'VANDERBILT'
        WHEN to_facility LIKE '%Saint Thomas' THEN 'SAINT THOMAS'
        WHEN to_facility LIKE '%HCA Health' THEN 'HCA'
        WHEN to_facility LIKE '%MAURY%' THEN 'MAURY'
        WHEN to_facility LIKE '%RIVERVIEW%' THEN 'RIVERVIEW MEDICAL'
        WHEN to_facility LIKE '%TROUSDALE%' THEN 'TROUSDALE'
        ELSE to_facility END
        AS system_name,
    to_facility,
    to_grouping,
    to_classification,
    COUNT(DISTINCT(to_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions,
    AVG(average_day_wait) AS avg_day_wait
FROM nash_referrals
WHERE to_classification LIKE 'General Acute Care Hospital'
GROUP BY system_name
ORDER BY total_patients DESC

"""

hospitals = pd.read_sql(query,db)

db.close()
hospitals.shape

(16, 8)

In [150]:
hospitals

,system_name,to_facility,to_grouping,to_classification,count_npi,total_patients,total_transactions,avg_day_wait
0,VANDERBILT,VANDERBILT UNIVERSITY MEDICAL CENTER,Hospitals,General Acute Care Hospital,3,441266,653863,18.237653
1,"HCA HEALTH SERVICES OF TENNESSEE, INC.","HCA HEALTH SERVICES OF TENNESSEE, INC.",Hospitals,General Acute Care Hospital,6,270573,362455,19.803941
2,SAINT THOMAS WEST HOSPITAL,SAINT THOMAS WEST HOSPITAL,Hospitals,General Acute Care Hospital,2,205062,247159,20.171423
3,MAURY,MAURY REGIONAL HOSPITAL,Hospitals,General Acute Care Hospital,1,96836,132087,22.322599
4,SAINT THOMAS RUTHERFORD HOSPITAL,SAINT THOMAS RUTHERFORD HOSPITAL,Hospitals,General Acute Care Hospital,1,77124,94125,19.944169
5,WILLIAMSON COUNTY HOSPITAL DISTRICT,WILLIAMSON COUNTY HOSPITAL DISTRICT,Hospitals,General Acute Care Hospital,1,71272,98355,23.414401
6,HTI MEMORIAL HOSPITAL CORPORATION,HTI MEMORIAL HOSPITAL CORPORATION,Hospitals,General Acute Care Hospital,1,69926,86255,19.026948
7,HENDERSONVILLE HOSPITAL CORPORATION,HENDERSONVILLE HOSPITAL CORPORATION,Hospitals,General Acute Care Hospital,1,48614,62984,21.799318
8,SUMNER REGIONAL MEDICAL CENTER LLC,SUMNER REGIONAL MEDICAL CENTER LLC,Hospitals,General Acute Care Hospital,1,46624,67847,22.021057
9,CENTRAL TENNESSEE HOSPITAL CORPORATION,CENTRAL TENNESSEE HOSPITAL CORPORATION,Hospitals,General Acute Care Hospital,1,37404,46920,16.894275


In [151]:
hospitals.system_name.tolist()

['VANDERBILT',
 'HCA HEALTH SERVICES OF TENNESSEE, INC.',
 'SAINT THOMAS WEST HOSPITAL',
 'MAURY',
 'SAINT THOMAS RUTHERFORD HOSPITAL',
 'WILLIAMSON COUNTY HOSPITAL DISTRICT',
 'HTI MEMORIAL HOSPITAL CORPORATION',
 'HENDERSONVILLE HOSPITAL CORPORATION',
 'SUMNER REGIONAL MEDICAL CENTER LLC',
 'CENTRAL TENNESSEE HOSPITAL CORPORATION',
 'NORTHCREST MEDICAL CENTER',
 'MACON COUNTY GENERAL HOSPITAL, INC.',
 'RIVERVIEW MEDICAL',
 'NASHVILLE GENERAL HOSPITAL',
 'SAINT THOMAS STONES RIVER HOSPITAL, LLC',
 'TROUSDALE']

In [20]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where

#WHERE to_classification LIKE '%Hospital%'



query = """
SELECT
    to_facility,
    to_grouping,
    to_classification,
    COUNT(DISTINCT(to_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions,
    AVG(average_day_wait) AS avg_day_wait
FROM nash_referrals
WHERE to_facility LIKE '%Vanderbilt%'
GROUP BY to_facility

"""

vandy = pd.read_sql(query,db)

db.close()

In [21]:
vandy

,to_facility,to_grouping,to_classification,count_npi,total_patients,total_transactions,avg_day_wait
0,VANDERBILT HEALTH AND WILLIAMSON MEDICAL CENTE...,Allopathic & Osteopathic Physicians,Internal Medicine,1,11027,11960,13.008615
1,"VANDERBILT HOME CARE SERVICES, LLC",Agencies,Home Health,1,241,344,23.545250
2,"VANDERBILT IMAGING SERVICES, LLC",Allopathic & Osteopathic Physicians,Radiology,2,6865,7822,25.855291
3,"VANDERBILT MAURY RADIATION ONCOLOGY, LLC",Ambulatory Health Care Facilities,Clinic/Center,1,168,537,0.779000
4,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,Hospitals,Rehabilitation Hospital,1,2832,2921,25.343882
5,VANDERBILT UNIVERSITY MEDICAL CENTER,Allopathic & Osteopathic Physicians,Internal Medicine,10,1027027,1568469,18.116048


#### Now add hospital CTE to extract 

In [ ]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as 
#Acute Hospital - CTE
#then select ALL combined associated facilities within 
#those hospital groups but excluding ancillary providers like 
#radiology, lab, path, etc.

query = """
WITH hospital_systems AS (
        SELECT
        CASE WHEN to_facility LIKE '%Vanderbilt%' THEN 'VANDERBILT'
            WHEN to_facility LIKE '%Saint Thomas' THEN 'SAINT THOMAS'
            WHEN to_facility LIKE '%HCA Health' THEN 'HCA'
            WHEN to_facility LIKE '%MAURY%' THEN 'MAURY'
            WHEN to_facility LIKE '%RIVERVIEW%' THEN 'RIVERVIEW MEDICAL'
            WHEN to_facility LIKE '%TROUSDALE%' THEN 'TROUSDALE'
            ELSE to_facility END
            AS system_name,
        to_facility,
        to_grouping,
        to_classification,
        to_npi,
        patient_count,
        transaction_count,
        average_day_wait
    FROM nash_referrals
    WHERE to_classification LIKE 'General Acute Care Hospital'
    ),
    
associated_facilities AS (
    SELECT
        CASE WHEN to_facility LIKE '%Vanderbilt%' THEN 'VANDERBILT'
            WHEN to_facility LIKE '%Saint Thomas' THEN 'SAINT THOMAS'
            WHEN to_facility LIKE '%HCA Health' THEN 'HCA'
            WHEN to_facility LIKE '%MAURY%' THEN 'MAURY'
            WHEN to_facility LIKE '%RIVERVIEW%' THEN 'RIVERVIEW MEDICAL'
            WHEN to_facility LIKE '%TROUSDALE%' THEN 'TROUSDALE'
            ELSE to_facility END
            AS system_name,
        to_facility,
        to_grouping,
        to_classification,
        to_npi,
        patient_count,
        transaction_count,
        average_day_wait
    FROM nash_referrals
    WHERE to_grouping NOT IN ("Managed Care Organizations",
                            "Nursing Service Providers",
                            "Physician Assistants & Advanced Practice Nursing Providers",
                            "Suppliers",
                            "Transportation Services",
                            "Managed Care Organizations",
                            "Dental Providers",
                            "Laboratories"
                            )
    AND to_classification NOT IN ("Radiology",
                                "Pathology",
                             "Anesthesiology",
                             "Emergency Medecine",
                             "Obstetrics & Gynecology",
                             "Pediatrics")
                             )

SELECT 
    associated_facilities.system_name AS system_name,
    COUNT(DISTINCT(associated_facilities.to_npi)) AS count_npi,
    SUM(associated_facilities.patient_count) AS total_patients,
    SUM(associated_facilities.transaction_count) AS total_transactions,
    AVG(associated_facilities.average_day_wait) AS avg_day_wait
FROM associated_facilities
INNER JOIN hospital_systems
ON associated_facilities.system_name = hospital_systems.system_name
GROUP BY associated_facilities.system_name

"""

test = pd.read_sql(query,db)

db.close()
test.shape

In [12]:
test['to_classification'].unique()

array(['General Acute Care Hospital'], dtype=object)

In [ ]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as 
#Acute Hospital - CTE
#then select ALL combined associated facilities within 
#those hospital groups but excluding ancillary providers like 
#radiology, lab, path, etc.

query = """
        SELECT
        CASE WHEN to_facility LIKE '%Vanderbilt%' THEN 'VANDERBILT'
            WHEN to_facility LIKE '%Saint Thomas' THEN 'SAINT THOMAS'
            WHEN to_facility LIKE '%HCA Health' THEN 'HCA'
            WHEN to_facility LIKE '%MAURY%' THEN 'MAURY'
            WHEN to_facility LIKE '%RIVERVIEW%' THEN 'RIVERVIEW MEDICAL'
            WHEN to_facility LIKE '%TROUSDALE%' THEN 'TROUSDALE'
            ELSE to_facility END
            AS system_name,
        to_facility,
        to_grouping,
        to_classification,
        to_npi,
        patient_count,
        transaction_count,
        average_day_wait
    FROM nash_referrals
    WHERE to_classification LIKE 'General Acute Care Hospital'
    ),
    
associated_facilities AS (
    SELECT
        CASE WHEN to_facility LIKE '%Vanderbilt%' THEN 'VANDERBILT'
            WHEN to_facility LIKE '%Saint Thomas' THEN 'SAINT THOMAS'
            WHEN to_facility LIKE '%HCA Health' THEN 'HCA'
            WHEN to_facility LIKE '%MAURY%' THEN 'MAURY'
            WHEN to_facility LIKE '%RIVERVIEW%' THEN 'RIVERVIEW MEDICAL'
            WHEN to_facility LIKE '%TROUSDALE%' THEN 'TROUSDALE'
            ELSE to_facility END
            AS system_name,
        to_facility,
        to_grouping,
        to_classification,
        to_npi,
        patient_count,
        transaction_count,
        average_day_wait
    FROM nash_referrals
    WHERE to_grouping NOT IN ("Managed Care Organizations",
                            "Nursing Service Providers",
                            "Physician Assistants & Advanced Practice Nursing Providers",
                            "Suppliers",
                            "Transportation Services",
                            "Managed Care Organizations",
                            "Dental Providers",
                            "Laboratories"
                            )
    AND to_classification NOT IN ("Radiology",
                                "Pathology",
                             "Anesthesiology",
                             "Emergency Medecine",
                             "Obstetrics & Gynecology",
                             "Pediatrics")
                             )

SELECT 
    associated_facilities.system_name AS system_name,
    COUNT(DISTINCT(associated_facilities.to_npi)) AS count_npi,
    SUM(associated_facilities.patient_count) AS total_patients,
    SUM(associated_facilities.transaction_count) AS total_transactions,
    AVG(associated_facilities.average_day_wait) AS avg_day_wait
FROM associated_facilities
INNER JOIN hospital_systems
ON associated_facilities.system_name = hospital_systems.system_name
GROUP BY associated_facilities.system_name

"""

test = pd.read_sql(query,db)

db.close()
test.shape

In [ ]:
test

#### Quick look at selected hospital 'brands' in Nashville CBSA

['VANDERBILT UNIVERSITY MEDICAL CENTER',
 'HCA HEALTH SERVICES OF TENNESSEE, INC.',
 'SAINT THOMAS WEST HOSPITAL',
 'MAURY REGIONAL HOSPITAL',
 'SAINT THOMAS RUTHERFORD HOSPITAL',
'WILLIAMSON COUNTY HOSPITAL DISTRICT',
'HTI MEMORIAL HOSPITAL CORPORATION',
 'HENDERSONVILLE HOSPITAL CORPORATION',
 'SUMNER REGIONAL MEDICAL CENTER LLC',
'CENTRAL TENNESSEE HOSPITAL CORPORATION',
 'NORTHCREST MEDICAL CENTER',
 'MACON COUNTY GENERAL HOSPITAL, INC.',
'RIVERVIEW MEDICAL CENTER LLC',
'NASHVILLE GENERAL HOSPITAL',
 'SAINT THOMAS STONES RIVER HOSPITAL, LLC',
 'TROUSDALE MEDICAL CENTER LLC']


In [13]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where

#WHERE to_classification LIKE '%Hospital%'

query = """
SELECT
    CASE WHEN to_facility LIKE '%Vanderbilt%' THEN 'Vanderbilt'
        WHEN to_facility LIKE '%Saint Thomas' THEN 'Saint Thomas'
        WHEN to_facility LIKE '%HCA Health' THEN 'HCA'
        WHEN to_facility LIKE '%MAURY%' THEN 'Maury'
        WHEN to_facility LIKE '%RIVERVIEW%' THEN 'Riverview Medical'
        WHEN to_facility LIKE '%TROUSDALE%' THEN 'Trousale'
        ELSE to_facility END
        AS hospital_system,
    to_facility,
    to_classification,
    COUNT(DISTINCT(to_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions,
    AVG(average_day_wait) AS avg_day_wait
FROM nash_referrals
WHERE to_classification NOT IN ("Radiology", 
                             "Anesthesiology", 
                             "Ambulance", 
                             "Emergency Medicine", 
                             "Pathology", 
                             "Durable Medical Equipment & Medical Supplies", 
                             "Clinical Medical Laboratory", 
                             "Portable X-ray and/or Other Portable Diagnostic Imaging Supplier", 
                             "Pharmacy", 
                             "Audiologist-Hearing Aid Fitter", 
                             "Prosthetic/Orthotic Supplier", 
                             "Eyewear Supplier", 
                             "Preferred Provider Organization",
                             "Nurse Anesthetist, Certified Registered")
GROUP BY hospital_system
ORDER BY total_patients DESC

"""

systems = pd.read_sql(query,db)

db.close()
systems.shape

(901, 7)

In [14]:
systems

,hospital_system,to_facility,to_classification,count_npi,total_patients,total_transactions,avg_day_wait
0,Vanderbilt,VANDERBILT UNIVERSITY MEDICAL CENTER,Internal Medicine,13,1035256,1573688,18.012606
1,SAINT THOMAS MEDICAL PARTNERS,SAINT THOMAS MEDICAL PARTNERS,Internal Medicine,1,398245,635587,25.089338
2,"HCA HEALTH SERVICES OF TENNESSEE, INC.","HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,9,276229,369833,19.606322
3,CENTENNIAL HEART LLC,CENTENNIAL HEART LLC,Specialist,1,227933,322911,30.705381
4,SAINT THOMAS WEST HOSPITAL,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,4,205510,247633,20.130065
...,...,...,...,...,...,...,...
896,"DAWES CHIROPRACTIC, PLLC","DAWES CHIROPRACTIC, PLLC",Chiropractor,1,12,116,0.000000
897,"AFFINITY HEALTHCARE, PLLC","AFFINITY HEALTHCARE, PLLC",Clinic/Center,1,12,57,0.000000
898,ROCK SPRINGS FAMILY CHIROPRACTIC,ROCK SPRINGS FAMILY CHIROPRACTIC,Chiropractor,1,11,128,0.000000
899,BANDALO CHIROPRACTIC PLLC,BANDALO CHIROPRACTIC PLLC,Chiropractor,1,11,72,0.000000


### Filter out unecessary classifications and groupings

In [78]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where

query = """
SELECT
    to_classification,
    to_grouping,
    COUNT(DISTINCT(to_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions,
    AVG(average_day_wait) AS avg_day_wait
FROM nash_referrals

GROUP BY to_classification
ORDER BY  to_grouping, total_patients DESC

"""

filter_class = pd.read_sql(query,db)

db.close()
filter_class.shape

(66, 6)

In [107]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where

query = """
SELECT
    from_classification,
    from_grouping,
    COUNT(DISTINCT(from_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions,
    AVG(average_day_wait) AS avg_day_wait
FROM nash_referrals

GROUP BY from_classification
ORDER BY  from_grouping, total_patients DESC

"""

filter_class = pd.read_sql(query,db)

db.close()
filter_class.shape

(55, 6)

In [108]:
filter_class

,from_classification,from_grouping,count_npi,total_patients,total_transactions,avg_day_wait
0,Internal Medicine,Allopathic & Osteopathic Physicians,1135,2096478,3556531,25.905800
1,Radiology,Allopathic & Osteopathic Physicians,270,1802013,2004985,29.763111
2,Pathology,Allopathic & Osteopathic Physicians,132,554681,778389,20.057731
3,Emergency Medicine,Allopathic & Osteopathic Physicians,208,329657,362521,22.551683
4,Family Medicine,Allopathic & Osteopathic Physicians,302,325199,665944,26.134990
5,Anesthesiology,Allopathic & Osteopathic Physicians,256,214845,283093,16.479608
6,Orthopaedic Surgery,Allopathic & Osteopathic Physicians,155,188616,339042,26.975842
7,Ophthalmology,Allopathic & Osteopathic Physicians,94,135132,235936,28.517717
8,Psychiatry & Neurology,Allopathic & Osteopathic Physicians,206,128418,221220,21.044007
9,Surgery,Allopathic & Osteopathic Physicians,156,124759,199861,20.928451


### Classifications of facilities

In [ ]:
WHERE to_grouping NOT IN ("Managed Care Organizations",
                            "Nursing Service Providers",
                            "Physician Assistants & Advanced Practice Nursing Providers",
                            "Suppliers",
                            "Transportation Services",
                            "Managed Care Organizations",
                            "Dental Providers",
                            "Laboratories"
                            )
AND to_classification NOT IN ("Radiology",
                                "Pathology",
                             "Anesthesiology",
                             "Emergency Medecine",
                             "Obstetrics & Gynecology",
                             "Pediatrics")

In [50]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where

query = """
SELECT
    to_classification,
    to_grouping,
    COUNT(DISTINCT(to_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions,
    AVG(average_day_wait) AS avg_day_wait
FROM nash_referrals

GROUP BY to_classification
ORDER BY  to_grouping, total_patients DESC

"""

classifications = pd.read_sql(query,db)

db.close()
classifications.shape

(66, 6)

In [51]:
classifications

,to_classification,to_grouping,count_npi,total_patients,total_transactions,avg_day_wait
0,Home Health,Agencies,26,10284,15303,27.847717
1,"Hospice Care, Community Based",Agencies,16,9791,21473,18.780531
2,Public Health or Welfare,Agencies,1,65,77,0.000000
3,Internal Medicine,Allopathic & Osteopathic Physicians,117,1786068,2904525,23.622803
4,Radiology,Allopathic & Osteopathic Physicians,18,1171861,1566927,26.387463
5,Pathology,Allopathic & Osteopathic Physicians,5,426114,564978,25.369043
6,Anesthesiology,Allopathic & Osteopathic Physicians,13,368285,479192,22.183574
7,Emergency Medicine,Allopathic & Osteopathic Physicians,25,284075,328993,29.161605
8,Family Medicine,Allopathic & Osteopathic Physicians,85,233302,469138,23.991478
9,Orthopaedic Surgery,Allopathic & Osteopathic Physicians,18,127850,243732,26.066921


In [56]:
fig = px.bar(classifications,
             x="to_grouping", 
             y="total_transactions", 
             #color="to_classification", 
             title="Referral Volume by Facility Type",
             #hover_name="country", 
             hover_data=["count_npi", "total_patients","total_transactions"])
fig.show()

In [80]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where

query = """
SELECT
    to_grouping,
    COUNT(DISTINCT(to_npi)) AS count_npi,
    SUM(patient_count) AS total_patients,
    SUM(transaction_count) AS total_transactions,
    AVG(average_day_wait) AS avg_day_wait
FROM nash_referrals
GROUP BY to_grouping
ORDER BY total_patients DESC

"""

groupings = pd.read_sql(query,db)

db.close()
groupings.shape

(66, 6)

In [79]:
groupings

,to_classification,to_grouping,count_npi,total_patients,total_transactions,avg_day_wait
0,Dermatology,Allopathic & Osteopathic Physicians,436,4764995,7207402,24.622381
1,General Acute Care Hospital,Hospitals,36,1430465,1953612,19.435539
2,Specialist,Other Service Providers,74,491077,761209,27.739854
3,Ambulance,Transportation Services,19,213330,238086,32.121362
4,Physiological Laboratory,Laboratories,17,181066,294136,28.757285
5,Clinic/Center,Ambulatory Health Care Facilities,170,151197,265403,14.685548
6,Durable Medical Equipment & Medical Supplies,Suppliers,68,89336,129440,29.863546
7,"Nurse Anesthetist, Certified Registered",Physician Assistants & Advanced Practice Nursi...,52,86059,153343,17.470257
8,Skilled Nursing Facility,Nursing & Custodial Care Facilities,72,44037,88812,20.578645
9,Home Health,Agencies,43,20140,36853,24.316040


In [ ]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

#select facility name where facility has to_classification as Acute Hospital - CTE
#then select ALL combined facility names where


groupings = pd.read_sql(query,db)

db.close()
groupings.shape